In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import datetime
import dateutil.parser
import time
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from tqdm import tqdm_notebook
pd.options.mode.chained_assignment = None

In [2]:
team_converter = pd.read_csv("source_files/team_converter.csv",encoding="cp1252")
big_five_league_fixtures = pd.read_csv("source_files/big_five_league_fixtures.csv")
big_five_league_fixtures["date"] = big_five_league_fixtures["date"].apply(lambda x:datetime.datetime.strptime(x,"%d/%m/%Y").date())
big_five_league_fixtures["home_points"] = big_five_league_fixtures["result"].map({"H":3,"D":1,"A":0})
big_five_league_fixtures["away_points"] = big_five_league_fixtures["result"].map({"A":3,"D":1,"H":0})

In [3]:
FIFA_league_season_teams_df = pd.read_csv("output_files/FIFA_league_season_teams_df.csv")
FIFA_team_season_players_df = pd.read_csv("output_files/FIFA_team_season_players_df.csv")
TM_team_managers_df = pd.read_csv("output_files/TM_team_managers_df.csv")

## Step 1: join team ids onto fixture data ##

In [4]:
FIFA_edition_dict = {
    "FIFA 2005":["08/10/2004","/fifa05_1/"],
    "FIFA 2006":["24/08/2005","/fifa06_2/"],
    "FIFA 2007":["28/08/2006","/fifa07_3/"],
    "FIFA 2008":["25/08/2007","/fifa08_4/"],
    "FIFA 2009":["01/09/2008","/fifa09_5/"],
    "FIFA 2010":["01/09/2009","/fifa10_6/"],
    "FIFA 2011":["01/09/2010","/fifa11_7/"],
    "FIFA 2012":["01/09/2011","/fifa12_8/"],
    "FIFA 2013":["31/08/2012","/fifa13_11/"],
    "FIFA 2014":["30/08/2013","/fifa14_12/"],
    "FIFA 2015":["29/08/2014","/fifa15_16/"],
    "FIFA 2016":["21/09/2015","/fifa16_19/"],
    "FIFA 2017":["25/08/2016","/fifa17_74/"],
    "FIFA 2018":["22/08/2017","/fifa18_174/"],
    "FIFA 2019":["21/08/2018","/fifa19_279/"],
}

In [5]:
big_five_league_fixtures["FIFA_edition"] = big_five_league_fixtures["season"].apply(lambda x: FIFA_edition_dict["FIFA "+str(x)][1])

In [6]:
big_five_league_fixtures = big_five_league_fixtures.join(team_converter.set_index("fixtures_team_name"),on="home_team",rsuffix="_dupe").drop("league_country_dupe",axis=1)
big_five_league_fixtures = big_five_league_fixtures.rename(index=str,columns={"TM_team_name":"home_TM_team_name","TM_team_id":"home_TM_team_id",
                                "FIFA_team_name":"home_FIFA_team_name","FIFA_team_id":"home_FIFA_team_id"})

In [7]:
big_five_league_fixtures = big_five_league_fixtures.join(team_converter.set_index("fixtures_team_name"),on="away_team",rsuffix="_dupe").drop("league_country_dupe",axis=1)
big_five_league_fixtures = big_five_league_fixtures.rename(index=str,columns={"TM_team_name":"away_TM_team_name","TM_team_id":"away_TM_team_id",
                                "FIFA_team_name":"away_FIFA_team_name","FIFA_team_id":"away_FIFA_team_id"})

## Step 2: join manager tenures onto fixture data ##

In [8]:
TM_team_managers_df["TM_manager_start_date"] = TM_team_managers_df["TM_manager_start_date"].apply(lambda x:dateutil.parser.parse(x))
TM_manager_end_date_cleaned_list = []
for d in TM_team_managers_df["TM_manager_end_date"]:
    try:
        TM_manager_end_date_cleaned_list.append(dateutil.parser.parse(d))
    except Exception:
        TM_manager_end_date_cleaned_list.append(datetime.datetime(2019,7,1))
TM_team_managers_df["TM_manager_end_date"] = TM_manager_end_date_cleaned_list

In [9]:
TM_manager_tenure_id_list = []
for i,r in tqdm_notebook(big_five_league_fixtures.iterrows(), total=big_five_league_fixtures.shape[0]):
    match_manager_list = []
    match_date = r["date"]
    home_TM_team_id = r["home_TM_team_id"]
    away_TM_team_id = r["away_TM_team_id"]
    try:
        home_manager_tenure_row = TM_team_managers_df[(TM_team_managers_df["TM_team_id"]==home_TM_team_id) & (TM_team_managers_df["TM_manager_start_date"]<match_date) & (TM_team_managers_df["TM_manager_end_date"]>=match_date)].iloc[0]
        match_manager_list.append([home_manager_tenure_row["TM_manager_id"],home_manager_tenure_row["TM_manager_name"],home_manager_tenure_row["TM_manager_tenure_id"]])
    except Exception:
        match_manager_list.append([None,None,None])
    try:
        away_manager_tenure_row = TM_team_managers_df[(TM_team_managers_df["TM_team_id"]==away_TM_team_id) & (TM_team_managers_df["TM_manager_start_date"]<match_date) & (TM_team_managers_df["TM_manager_end_date"]>=match_date)].iloc[0]
        match_manager_list.append([away_manager_tenure_row["TM_manager_id"],away_manager_tenure_row["TM_manager_name"],away_manager_tenure_row["TM_manager_tenure_id"]])
    except Exception:
        match_manager_list.append([None,None,None])
    TM_manager_tenure_id_list.append(match_manager_list)

In [10]:
TM_manager_tenure_id_list = [m[0] + m[1] for m in TM_manager_tenure_id_list]
big_five_league_expected_points_manager_tenures_df = pd.concat([big_five_league_fixtures.reset_index(drop=True),
            pd.DataFrame(TM_manager_tenure_id_list,columns=["home_TM_manager_id","home_TM_manager_name","home_TM_manager_tenure_id",
                                                           "away_TM_manager_id","away_TM_manager_name","away_TM_manager_tenure_id"])],axis=1)

## Step 3: join expected points onto fixture data ##

In [11]:
expected_points = pd.read_csv("output_files/expected_points_df.csv",encoding="cp1252")

In [12]:
big_five_league_expected_points_manager_tenures_df["exp_pts_join"] = big_five_league_expected_points_manager_tenures_df["season"].astype(str) + big_five_league_expected_points_manager_tenures_df["home_team"] + big_five_league_expected_points_manager_tenures_df["away_team"] 

In [13]:
expected_points["exp_pts_join"] = expected_points["season"].astype(str) + expected_points["home_team"] + expected_points["away_team"]
expected_points.index = expected_points["exp_pts_join"]
expected_points = expected_points[["HomeExpectedPoints","AwayExpectedPoints"]]

In [14]:
big_five_league_expected_points_manager_tenures_df = big_five_league_expected_points_manager_tenures_df.join(expected_points,on="exp_pts_join")

In [15]:
big_five_league_expected_points_manager_tenures_df.drop("exp_pts_join",axis=1,inplace=True)
big_five_league_expected_points_manager_tenures_df.rename(columns={"HomeExpectedPoints":"home_expected_points","AwayExpectedPoints":"away_expected_points"},inplace=True)

In [16]:
big_five_league_expected_points_manager_tenures_df.to_csv("output_files/big_five_league_expected_points_manager_tenures_df.csv")
big_five_league_expected_points_manager_tenures_df

,league_country,season,date,home_team,away_team,home_goals,away_goals,result,home_points,away_points,...,away_FIFA_team_name,away_FIFA_team_id,home_TM_manager_id,home_TM_manager_name,home_TM_manager_tenure_id,away_TM_manager_id,away_TM_manager_name,away_TM_manager_tenure_id,home_expected_points,away_expected_points
0,England,2005,2004-08-14,Aston Villa,Southampton,2.0,0.0,H,3.0,0.0,...,southampton,17,446.0,David O'Leary,446_1,1583.0,Paul Sturrock,1583_1,1.719966,1.006168
1,England,2005,2004-08-14,Blackburn,West Brom,1.0,1.0,D,1.0,1.0,...,west-brom,109,449.0,Graeme Souness,449_4,600.0,Gary Megson,600_4,1.773376,0.938275
2,England,2005,2004-08-14,Bolton,Charlton,4.0,1.0,H,3.0,0.0,...,charlton-athletic,89,445.0,Sam Allardyce,445_2,998.0,Alan Curbishley,998_1,1.585781,1.127233
3,England,2005,2004-08-14,Man City,Fulham,1.0,1.0,D,1.0,1.0,...,fulham,144,453.0,Kevin Keegan,453_3,451.0,Chris Coleman,451_1,1.480450,1.195853
4,England,2005,2004-08-14,Middlesbrough,Newcastle,2.0,2.0,D,1.0,1.0,...,newcastle-united,13,458.0,Steve McClaren,458_1,461.0,Sir Bobby Robson,461_3,1.485955,1.228854
5,England,2005,2004-08-14,Norwich,Crystal Palace,1.0,1.0,D,1.0,1.0,...,crystal-palace,1799,605.0,Nigel Worthington,605_2,1598.0,Iain Dowie,1598_1,1.724511,0.997613
6,England,2005,2004-08-14,Portsmouth,Birmingham,1.0,1.0,D,1.0,1.0,...,birmingham-city,88,448.0,Harry Redknapp,448_3,447.0,Steve Bruce,447_5,1.589952,1.130814
7,England,2005,2004-08-14,Tottenham,Liverpool,1.0,1.0,D,1.0,1.0,...,liverpool,9,1647.0,Jacques Santini,1647_6,1522.0,Rafael Benítez,1522_5,1.157328,1.587353
8,England,2005,2004-08-15,Chelsea,Man United,1.0,0.0,H,3.0,0.0,...,manchester-united,11,781.0,José Mourinho,781_1,4.0,Sir Alex Ferguson,4_1,1.663199,1.101113
9,England,2005,2004-08-15,Everton,Arsenal,1.0,4.0,A,0.0,3.0,...,arsenal,1,450.0,David Moyes,450_1,280.0,Arsène Wenger,280_3,0.613474,2.188270


## Step 4: create dataframe of each team's performance in each season ##

In [17]:
team_season_performance_list = []

for c in big_five_league_expected_points_manager_tenures_df["league_country"].unique():
    league_country_df = big_five_league_expected_points_manager_tenures_df[big_five_league_expected_points_manager_tenures_df["league_country"]==c]
    seasons = league_country_df["season"].unique()

    for s in seasons:
        season_df = league_country_df[league_country_df["season"]==s]
        for t in season_df["home_team"].unique():
            home_games = season_df[(season_df["home_team"]==t) & (season_df["home_points"]>=0)]
            away_games = season_df[(season_df["away_team"]==t) & (season_df["away_points"]>=0)]
            total_games = home_games.shape[0] + away_games.shape[0]
            home_points = np.sum(home_games["home_points"])
            away_points = np.sum(away_games["away_points"])
            total_points = home_points + away_points
            points_per_game = total_points / total_games
            home_expected_points = np.sum(home_games["home_expected_points"])
            away_expected_points = np.sum(away_games["away_expected_points"])
            total_expected_points = home_expected_points + away_expected_points
            expected_points_per_game = total_expected_points / total_games
            points_per_game_above_expectation = points_per_game - expected_points_per_game

            team_season_performance_list.append([c,s,t,total_games,total_points,total_expected_points,
                                                 points_per_game,expected_points_per_game,points_per_game_above_expectation])

team_season_performance_df = pd.DataFrame(team_season_performance_list,columns=["league_country","season","team",
                                 "total_games","total_points","total_expected_points","points_per_game","expected_points_per_game",
                                 "points_per_game_above_expectation"])

team_season_performance_df.sort_values("points_per_game_above_expectation",ascending=False,inplace=True)
team_season_performance_df.reset_index(drop=True,inplace=True)

In [18]:
team_season_performance_df.to_csv("output_files/team_season_performance_df.csv")
team_season_performance_df

,league_country,season,team,total_games,total_points,total_expected_points,points_per_game,expected_points_per_game,points_per_game_above_expectation
0,England,2016,Leicester,38,81.0,40.185698,2.131579,1.057518,1.074061
1,Germany,2017,RB Leipzig,34,67.0,35.656922,1.970588,1.048733,0.921855
2,England,2019,Liverpool,20,54.0,36.666250,2.700000,1.833312,0.866688
3,Germany,2019,Dortmund,17,42.0,27.708397,2.470588,1.629906,0.840683
4,France,2012,Montpellier,38,82.0,50.774566,2.157895,1.336173,0.821722
5,France,2017,Monaco,38,95.0,64.264993,2.500000,1.691184,0.808816
6,Italy,2017,Atalanta,38,72.0,44.159112,1.894737,1.162082,0.732655
7,France,2010,Montpellier,38,69.0,42.855819,1.815789,1.127785,0.688005
8,Germany,2011,Dortmund,34,75.0,51.829176,2.205882,1.524388,0.681495
9,Spain,2006,Osasuna,38,68.0,42.436244,1.789474,1.116743,0.672730


## Step 5: create dataframe of each manager tenure, with past and future performance ##

In [19]:
manager_tenure_performance_list = []
for i in tqdm_notebook(TM_team_managers_df["TM_manager_tenure_id"].unique()):
    try:
        home_games = len(big_five_league_expected_points_manager_tenures_df[(big_five_league_expected_points_manager_tenures_df["home_TM_manager_tenure_id"]==i) & (big_five_league_expected_points_manager_tenures_df["home_points"]>=0)]["home_points"])
        home_points = np.sum(big_five_league_expected_points_manager_tenures_df[big_five_league_expected_points_manager_tenures_df["home_TM_manager_tenure_id"]==i]["home_points"])
        home_expected_points = np.sum(big_five_league_expected_points_manager_tenures_df[(big_five_league_expected_points_manager_tenures_df["home_TM_manager_tenure_id"]==i) & (big_five_league_expected_points_manager_tenures_df["home_points"]>=0)]["home_expected_points"])
        away_games = len(big_five_league_expected_points_manager_tenures_df[(big_five_league_expected_points_manager_tenures_df["away_TM_manager_tenure_id"]==i) & (big_five_league_expected_points_manager_tenures_df["away_points"]>=0)]["away_points"])
        away_points = np.sum(big_five_league_expected_points_manager_tenures_df[big_five_league_expected_points_manager_tenures_df["away_TM_manager_tenure_id"]==i]["away_points"])
        away_expected_points = np.sum(big_five_league_expected_points_manager_tenures_df[(big_five_league_expected_points_manager_tenures_df["away_TM_manager_tenure_id"]==i) & (big_five_league_expected_points_manager_tenures_df["away_points"]>=0)]["away_expected_points"])
        total_games = home_games + away_games
        total_points = home_points + away_points
        total_expected_points = home_expected_points + away_expected_points
        total_points_per_game = total_points / total_games
        total_expected_points_per_game = total_expected_points / total_games
        points_per_game_above_expectation = total_points_per_game - total_expected_points_per_game
        manager_tenure_performance_list.append([total_games,total_points,total_expected_points,total_points_per_game,total_expected_points_per_game,points_per_game_above_expectation])
    except Exception:
        pass
        manager_tenure_performance_list.append([None,None,None,None,None,None])

In [20]:
manager_tenure_performance_df = pd.concat([TM_team_managers_df,pd.DataFrame(manager_tenure_performance_list,columns=["current_tenure_total_games","current_tenure_total_points","current_tenure_total_expected_points",
                                                        "current_tenure_points_per_game","current_tenure_expected_points_per_game","current_tenure_points_per_game_above_expectation"])],axis=1)
manager_tenure_performance_df = manager_tenure_performance_df[manager_tenure_performance_df["current_tenure_total_games"]>0]
manager_tenure_performance_df.drop("Unnamed: 0",axis=1,inplace=True)
manager_tenure_performance_df.sort_values("TM_manager_start_date",inplace=True)
manager_tenure_performance_df.reset_index(drop=True,inplace=True)
manager_tenure_performance_df

,TM_team_name,TM_team_id,TM_manager_link,TM_manager_name,TM_manager_id,TM_manager_date_of_birth,TM_manager_nationality,TM_manager_start_date,TM_manager_end_date,TM_manager_tenure_id,current_tenure_total_games,current_tenure_total_points,current_tenure_total_expected_points,current_tenure_points_per_game,current_tenure_expected_points_per_game,current_tenure_points_per_game_above_expectation
0,manchester-united,985,/sir-alex-ferguson/profil/trainer/4,Sir Alex Ferguson,4,1941-12-31,Scotland,1986-11-06,2013-06-30,4_1,342.0,769.0,714.478314,2.248538,2.089118,0.159420
1,sc-freiburg,60,/volker-finke/profil/trainer/65,Volker Finke,65,1948-03-24,Germany,1991-07-01,2007-06-30,65_1,34.0,18.0,38.097911,0.529412,1.120527,-0.591115
2,charlton-athletic,358,/alan-curbishley/profil/trainer/998,Alan Curbishley,998,1957-11-08,England,1991-07-24,2006-06-30,998_1,76.0,93.0,92.753268,1.223684,1.220438,0.003246
3,fc-arsenal,11,/arsene-wenger/profil/trainer/280,Arsène Wenger,280,1949-10-22,France,1996-10-01,2018-06-30,280_3,532.0,1022.0,1012.975420,1.921053,1.904089,0.016963
4,deportivo-la-coruna,897,/javier-irureta/profil/trainer/1556,Javier Irureta,1556,1948-04-01,Spain,1998-07-01,2005-06-01,1556_5,38.0,51.0,64.233019,1.342105,1.690343,-0.348237
5,sv-werder-bremen,86,/thomas-schaaf/profil/trainer/224,Thomas Schaaf,224,1961-04-30,Germany,1999-05-10,2013-05-15,224_1,305.0,484.0,488.334182,1.586885,1.601096,-0.014210
6,newcastle-united,762,/sir-bobby-robson/profil/trainer/461,Sir Bobby Robson,461,1933-02-18,England,1999-09-01,2004-08-29,461_3,4.0,2.0,6.605146,0.500000,1.651286,-1.151286
7,bolton-wanderers,355,/sam-allardyce/profil/trainer/445,Sam Allardyce,445,1954-10-19,England,1999-10-19,2007-04-30,445_2,112.0,169.0,136.178261,1.508929,1.215877,0.293051
8,sheffield-united,350,/neil-warnock/profil/trainer/1527,Neil Warnock,1527,1948-12-01,England,1999-12-02,2007-05-15,1527_2,38.0,38.0,40.646798,1.000000,1.069653,-0.069653
9,west-bromwich-albion,984,/gary-megson/profil/trainer/600,Gary Megson,600,1959-05-02,England,2000-03-09,2004-10-26,600_4,10.0,8.0,11.642484,0.800000,1.164248,-0.364248


In [21]:
manager_previous_tenure_performance_list = []
manager_all_previous_tenures_performance_list = []
manager_all_future_tenures_performance_list = []
for i,r in tqdm_notebook(manager_tenure_performance_df.iterrows(),total=manager_tenure_performance_df.shape[0]):
    TM_manager_id = r["TM_manager_id"]
    TM_manager_start_date = r["TM_manager_start_date"]
    all_previous_tenures = manager_tenure_performance_df[(manager_tenure_performance_df["TM_manager_id"]==TM_manager_id)&(manager_tenure_performance_df["TM_manager_start_date"]<TM_manager_start_date)]
    all_future_tenures = manager_tenure_performance_df[(manager_tenure_performance_df["TM_manager_id"]==TM_manager_id)&(manager_tenure_performance_df["TM_manager_end_date"]>TM_manager_start_date)]
    try:
        previous_tenure = list(all_previous_tenures.iloc[-1])[10:]
        manager_previous_tenure_performance_list.append(previous_tenure)
    
        all_previous_total_games = np.sum(all_previous_tenures["current_tenure_total_games"])
        all_previous_total_points = np.sum(all_previous_tenures["current_tenure_total_points"])
        all_previous_total_expected_points = np.sum(all_previous_tenures["current_tenure_total_expected_points"])
        all_previous_points_per_game = all_previous_total_points / all_previous_total_games
        all_previous_expected_points_per_game = all_previous_total_expected_points / all_previous_total_games
        all_previous_points_per_game_above_expectation = all_previous_points_per_game - all_previous_expected_points_per_game 
        manager_all_previous_tenures_performance_list.append([all_previous_total_games,all_previous_total_points,all_previous_total_expected_points,all_previous_points_per_game,all_previous_expected_points_per_game,all_previous_points_per_game_above_expectation]) 
    except Exception:
        manager_previous_tenure_performance_list.append([None,None,None,None,None,None])
        manager_all_previous_tenures_performance_list.append([None,None,None,None,None,None])
        
    try:
        all_future_total_games = np.sum(all_future_tenures["current_tenure_total_games"])
        all_future_total_points = np.sum(all_future_tenures["current_tenure_total_points"])
        all_future_total_expected_points = np.sum(all_future_tenures["current_tenure_total_expected_points"])
        all_future_points_per_game = all_future_total_points / all_future_total_games
        all_future_expected_points_per_game = all_future_total_expected_points / all_future_total_games
        all_future_points_per_game_above_expectation = all_future_points_per_game - all_future_expected_points_per_game
        manager_all_future_tenures_performance_list.append([all_future_total_games,all_future_total_points,all_future_total_expected_points,all_future_points_per_game,all_future_expected_points_per_game,all_future_points_per_game_above_expectation])
    except Exception:
        manager_all_future_tenures_performance_list.append([None,None,None,None,None,None])

In [22]:
manager_previous_tenure_performance_df = pd.DataFrame(manager_previous_tenure_performance_list,
                                                      columns=["previous_tenure_total_games","previous_tenure_total_points","previous_tenure_total_expected_points",
                                                        "previous_tenure_points_per_game","previous_tenure_expected_points_per_game","previous_tenure_points_per_game_above_expectation"])

manager_all_previous_tenures_performance_df = pd.DataFrame(manager_all_previous_tenures_performance_list,
                                                      columns=["all_previous_tenures_total_games","all_previous_tenures_total_points","all_previous_tenures_total_expected_points",
                                                        "all_previous_tenures_points_per_game","all_previous_tenures_expected_points_per_game","all_previous_tenures_points_per_game_above_expectation"])

manager_all_future_tenures_performance_df = pd.DataFrame(manager_all_future_tenures_performance_list,
                                                      columns=["all_future_tenures_total_games","all_future_tenures_total_points","all_future_tenures_total_expected_points",
                                                        "all_future_tenures_points_per_game","all_future_tenures_expected_points_per_game","all_future_tenures_points_per_game_above_expectation"])

In [23]:
manager_all_tenures_performance_df = pd.concat([manager_tenure_performance_df,manager_previous_tenure_performance_df,manager_all_previous_tenures_performance_df,manager_all_future_tenures_performance_df],axis=1)

In [24]:
manager_all_tenures_performance_df.to_csv("output_files/manager_all_tenures_performance_df.csv")
manager_all_tenures_performance_df

,TM_team_name,TM_team_id,TM_manager_link,TM_manager_name,TM_manager_id,TM_manager_date_of_birth,TM_manager_nationality,TM_manager_start_date,TM_manager_end_date,TM_manager_tenure_id,...,all_previous_tenures_total_expected_points,all_previous_tenures_points_per_game,all_previous_tenures_expected_points_per_game,all_previous_tenures_points_per_game_above_expectation,all_future_tenures_total_games,all_future_tenures_total_points,all_future_tenures_total_expected_points,all_future_tenures_points_per_game,all_future_tenures_expected_points_per_game,all_future_tenures_points_per_game_above_expectation
0,manchester-united,985,/sir-alex-ferguson/profil/trainer/4,Sir Alex Ferguson,4,1941-12-31,Scotland,1986-11-06,2013-06-30,4_1,...,NaN,NaN,NaN,NaN,342.0,769.0,714.478314,2.248538,2.089118,0.159420
1,sc-freiburg,60,/volker-finke/profil/trainer/65,Volker Finke,65,1948-03-24,Germany,1991-07-01,2007-06-30,65_1,...,NaN,NaN,NaN,NaN,34.0,18.0,38.097911,0.529412,1.120527,-0.591115
2,charlton-athletic,358,/alan-curbishley/profil/trainer/998,Alan Curbishley,998,1957-11-08,England,1991-07-24,2006-06-30,998_1,...,NaN,NaN,NaN,NaN,138.0,175.0,172.447975,1.268116,1.249623,0.018493
3,fc-arsenal,11,/arsene-wenger/profil/trainer/280,Arsène Wenger,280,1949-10-22,France,1996-10-01,2018-06-30,280_3,...,NaN,NaN,NaN,NaN,532.0,1022.0,1012.975420,1.921053,1.904089,0.016963
4,deportivo-la-coruna,897,/javier-irureta/profil/trainer/1556,Javier Irureta,1556,1948-04-01,Spain,1998-07-01,2005-06-01,1556_5,...,NaN,NaN,NaN,NaN,59.0,68.0,92.043238,1.152542,1.560055,-0.407513
5,sv-werder-bremen,86,/thomas-schaaf/profil/trainer/224,Thomas Schaaf,224,1961-04-30,Germany,1999-05-10,2013-05-15,224_1,...,NaN,NaN,NaN,NaN,350.0,530.0,543.518930,1.514286,1.552911,-0.038626
6,newcastle-united,762,/sir-bobby-robson/profil/trainer/461,Sir Bobby Robson,461,1933-02-18,England,1999-09-01,2004-08-29,461_3,...,NaN,NaN,NaN,NaN,4.0,2.0,6.605146,0.500000,1.651286,-1.151286
7,bolton-wanderers,355,/sam-allardyce/profil/trainer/445,Sam Allardyce,445,1954-10-19,England,1999-10-19,2007-04-30,445_2,...,NaN,NaN,NaN,NaN,398.0,523.0,474.878584,1.314070,1.193162,0.120908
8,sheffield-united,350,/neil-warnock/profil/trainer/1527,Neil Warnock,1527,1948-12-01,England,1999-12-02,2007-05-15,1527_2,...,NaN,NaN,NaN,NaN,94.0,88.0,96.517449,0.936170,1.026781,-0.090611
9,west-bromwich-albion,984,/gary-megson/profil/trainer/600,Gary Megson,600,1959-05-02,England,2000-03-09,2004-10-26,600_4,...,NaN,NaN,NaN,NaN,94.0,99.0,109.215528,1.053191,1.161867,-0.108676


## Step 6: create decision-tree dataframe for manager tenures ##

In [25]:
manager_decision_tree_df = manager_all_tenures_performance_df[["TM_manager_name","TM_manager_id","TM_manager_start_date","TM_manager_tenure_id","current_tenure_total_games","current_tenure_points_per_game_above_expectation"]]
manager_decision_tree_df = manager_decision_tree_df[manager_decision_tree_df["current_tenure_total_games"]>=15]
manager_decision_tree_df.sort_values(["TM_manager_id","TM_manager_start_date"],inplace=True)
manager_decision_tree_df.reset_index(inplace=True,drop=True)
manager_decision_tree_df

,TM_manager_name,TM_manager_id,TM_manager_start_date,TM_manager_tenure_id,current_tenure_total_games,current_tenure_points_per_game_above_expectation
0,Dick Advocaat,3,2004-11-02,3_1,18.0,-0.245145
1,Dick Advocaat,3,2015-03-17,3_2,17.0,-0.322931
2,Sir Alex Ferguson,4,1986-11-06,4_1,342.0,0.159420
3,Klaus Augenthaler,10,2003-05-13,10_2,38.0,0.068748
4,Klaus Augenthaler,10,2005-12-29,10_3,51.0,-0.308582
5,Jörg Berger,16,2004-11-17,16_11,21.0,-0.093710
6,Rudi Bommer,19,2006-07-01,19_2,34.0,-0.243650
7,Christoph Daum,38,2006-11-27,38_4,34.0,-0.031555
8,Volker Finke,65,1991-07-01,65_1,34.0,-0.591115
9,Friedhelm Funkel,72,2004-07-01,72_4,136.0,-0.094684


In [26]:
def get_sequence_overperformance(manager_id):
    manager_sequence_overperformance = []
    all_tenures = manager_all_tenures_performance_df[(manager_all_tenures_performance_df["TM_manager_id"]==manager_id) & (manager_all_tenures_performance_df["current_tenure_total_games"]>=15)]
    manager_sequence_overperformance.append(all_tenures["TM_manager_name"].iloc[0])
    for i in range(5):
        try:
            selected_tenure = all_tenures.iloc[i]
            selected_tenure_points_per_game_above_expectation = selected_tenure["current_tenure_points_per_game_above_expectation"]
            if selected_tenure_points_per_game_above_expectation > 0:
                selected_tenure_overperformance = 1
            else:
                selected_tenure_overperformance = 0
            manager_sequence_overperformance = manager_sequence_overperformance + [selected_tenure_overperformance,selected_tenure_points_per_game_above_expectation]
        except Exception:
            manager_sequence_overperformance = manager_sequence_overperformance + [None,None]
    return manager_sequence_overperformance

In [27]:
manager_sequence_overperformance_list = []
for i in manager_decision_tree_df["TM_manager_id"].unique():
    manager_sequence_overperformance_list.append(get_sequence_overperformance(i))
manager_sequence_overperformance_df = pd.DataFrame(manager_sequence_overperformance_list,
                                    columns=["TM_manager_name","tenure1_overperformance","tenure1_above_expectation",
                                            "tenure2_overperformance","tenure2_above_expectation",
                                            "tenure3_overperformance","tenure3_above_expectation",
                                            "tenure4_overperformance","tenure4_above_expectation",
                                            "tenure5_overperformance","tenure5_above_expectation"])
manager_sequence_overperformance_df

,TM_manager_name,tenure1_overperformance,tenure1_above_expectation,tenure2_overperformance,tenure2_above_expectation,tenure3_overperformance,tenure3_above_expectation,tenure4_overperformance,tenure4_above_expectation,tenure5_overperformance,tenure5_above_expectation
0,Dick Advocaat,0,-0.245145,0.0,-0.322931,NaN,NaN,NaN,NaN,NaN,NaN
1,Sir Alex Ferguson,1,0.159420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Klaus Augenthaler,1,0.068748,0.0,-0.308582,NaN,NaN,NaN,NaN,NaN,NaN
3,Jörg Berger,0,-0.093710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Rudi Bommer,0,-0.243650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Christoph Daum,0,-0.031555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Volker Finke,0,-0.591115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Friedhelm Funkel,0,-0.094684,0.0,-0.414625,0.0,-0.005051,NaN,NaN,NaN,NaN
8,Falko Götz,0,-0.004882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Jupp Heynckes,0,-0.481258,1.0,0.282919,1.0,0.149954,1.0,0.157568,NaN,NaN


In [28]:
manager_sequence_overperformance_df.to_csv("output_files/manager_sequence_overperformance_df.csv")